# functools

> The [functools](https://docs.python.org/3.8/library/functools.html) module is for higher-order functions: functions that act on or return other functions. In general, any callable object can be treated as a function for the purposes of this module.

***

## functools.cached_property

`cached_property` is applied to a method of a class that is normally expensive to compute but **also doesn't change throughout the lifetime of the object**. The function turns a method into a property and caches the results the first time the method is called. Every subsequent time the method is called the cached results are returned.

In [2]:
from functools import cached_property

In [3]:
class LongList:
    
    def __init__(self, data):
        self._data = data
        
    @cached_property
    def maximum(self):
        print('Only prints first time')
        return max(self._data)
    
    @cached_property
    def minimum(self):
        print('Only prints first time')
        return min(self._data)

In [4]:
import random

ls = LongList([random.randint(5, 500) for _ in range(1_000_000)])

print(ls.maximum, ls.minimum, sep='\t', end='\n\n')
print(ls.maximum, ls.minimum, sep='\t')

# Even when you change the underlying data structure
# that minimum and maximum work on
ls._data = [random.randint(100, 300) for _ in range(1_000_000)]
# minimum and maximum don't recompute 
print(ls.maximum, ls.minimum, sep='\t')

Only prints first time
Only prints first time
500	5

500	5
500	5


## functools.cmp_to_key

Transforms a comparison function into a [key function](https://docs.python.org/3.8/glossary.html#term-key-function). \
A comparison function is one that takes in two arguments, call them `a` and `b`, and returns -1 if a < b, 0 if a == b, or 1 if a > b.

In [5]:
from functools import cmp_to_key

In [6]:
@cmp_to_key
def compare(a, b):
    if a < b:
        return -1
    elif a > b:
        return 1
    return 0

In [7]:
ls = [random.randint(1, 50) for _ in range(50)]

print(ls)
print()

print(sorted(ls, key=compare))

[49, 8, 43, 45, 14, 23, 9, 49, 41, 23, 23, 1, 28, 46, 15, 24, 43, 11, 16, 5, 10, 11, 8, 25, 42, 14, 1, 27, 21, 26, 44, 40, 39, 2, 41, 22, 16, 18, 28, 33, 46, 47, 8, 43, 24, 13, 33, 17, 22, 24]

[1, 1, 2, 5, 8, 8, 8, 9, 10, 11, 11, 13, 14, 14, 15, 16, 16, 17, 18, 21, 22, 22, 23, 23, 23, 24, 24, 24, 25, 26, 27, 28, 28, 33, 33, 39, 40, 41, 41, 42, 43, 43, 43, 44, 45, 46, 46, 47, 49, 49]


## lru_cache

Saves the `maxsize` most recent calls of a function for memoization. It is used to save time and computation on function calls with the same arguments multiple times.

We'll use the fibonacci sequence to illustrate the performance boost.

In [8]:
from functools import lru_cache

In [9]:
def fib(n):
    """Return the nth fibonacci number"""
    if n <= 1:
        return 1
    return fib(n-1) + fib(n-2)

In [10]:
# takes around 5 - 6 seconds on a 2.6 GHz Intel Core i5
%time fib(36)

CPU times: user 5.86 s, sys: 8.43 ms, total: 5.87 s
Wall time: 5.95 s


24157817

In [11]:
@lru_cache
def fib(n):
    """Return the nth fibonacci number"""
    if n <= 1:
        return 1
    return fib(n-1) + fib(n-2)

In [12]:
%time fib(36)

CPU times: user 33 µs, sys: 7 µs, total: 40 µs
Wall time: 42 µs


24157817

without lru_cache computing the 36<sup>th</sup> fibonacci takes around 6 seconds while with it computing it takes only around 40 microseconds. That's a 150,000 times speed-up!

## total_ordering

If a user defined class implements operations such as obj1 > obj2, known as rich comparison ordering methods then as long as you implement `__eq__` along with one of `__le__`, `__ge__`, `__lt__`, `__gt__` you don't have to implement the rest. It's done for us.

Note from the documentation:
> While this decorator makes it easy to create well behaved totally ordered types, it does come at the cost of slower execution and more complex stack traces for the derived comparison methods. If performance benchmarking indicates this is a bottleneck for a given application, implementing all six rich comparison methods instead is likely to provide an easy speed boost.

In [13]:
from functools import total_ordering

In [14]:
from math import isclose
# we'l define an object that can be compared with others of its type
@total_ordering
class Area:
    
    def __init__(self, length, width):
        self.length = length
        self.width = width
        
    @property
    def area(self):
        return self.length * self.width
    
    def __repr__(self):
        return f'Area({self.length}, {self.width})'
    
    def __str__(self):
        return f'Area: {self.area} sqft.'
    
    def __eq__(self, other):
        return isclose(self.area, other.area)
    
    def __lt__(self, other):
        return self.area < other.area

In [15]:
a1 = Area(32, 3)
print(a1)
a2 = Area(8, 13)
print(a2)

Area: 96 sqft.
Area: 104 sqft.


In [16]:
print(a1 == a2)
print(a1 < a2)
print(a1 <= a2)

print(a1 > a2)
print(a1 >= a2)

False
True
True
False
False


# partial

If your'e familiar with functional programming `partial` allows you to take in a function and return a function where some of its parameters are filled in. This lets us dynamically create functions that are all based on one, but all can perform slightly different tasks.

Here's an exceedingly stupid example.

In [17]:
from functools import partial

In [18]:
def add(a, b):
    return a + b

add1 = partial(add, 1)
add5 = partial(add, 5)
add10 = partial(add, 10)

In [19]:
add1(5), add5(5), add10(5)

(6, 10, 15)

# partialmethod

Works like `partial` except only on class methods not basic functions.

In [20]:
from functools import partialmethod

In [44]:
class MyMath:

    def add(self, a, b):
        return a + b
    
    add1 = partialmethod(add, 1)
    
    def sub(self, a, b):
        return a - b
    
    sub1 = partialmethod(sub, b=1)
    

In [54]:
math = MyMath()

print(math.add(5, 6), math.add1(6), math.sub(5, 6), math.sub1(1), sep='\t')


11	7	-1	0
